# Amirali Amini
## 610399102
### BIC HW1

To expedite the implementation of the algorithm, I utilized the TamPy library.

In [2]:
import random
import numpy as np



In this algorithm, during each generation of the new population, 50 samples are taken, with 20 members being selected twice. From these 20 members, one is chosen based on merit. Consequently, the population must always exceed 20 members. 

The purpose of this function is to reduce pressure on the population, as this problem features local maxima and does not possess a single absolute maximum. For mutation, I have implemented mechanisms that apply probabilities ranging from 0.2 to 0.3 for values less than 60. For values between 60 and 100, I tested mutation rates between 0.1 and 0.7. 

Among these, a mutation rate of 0.7 yielded the best results for 60, while a rate of 0.5 was used for 100. Given that the algorithm is time-intensive, I opted to maintain this value.

Furthermore, the mutation algorithm operates by swapping a maximum of 20% of the members without generating duplicate values within a chromosome. This approach guarantees that each chromosome remains free of redundant genes. 

To ensure this principle is upheld when producing a new offspring, I check that the genome of the offspring does not contain duplicate genes inherited from the parents. For crossover, I select two random points and generate offspring from these positions.

Regarding the population size, smaller numbers do not significantly impact the results; however, I achieved the fastest solutions with 28 for 8 and 48 for 20. 

For the case of 60, I initially tested with a population of 60 but observed that all the members became very similar, and mutations occurred infrequently and slowly. Subsequently, I tested a population of 120 to enhance variability and increase the chances of reaching local maxima.

The fitness function is designed to count the number of conflicts among the pieces on the board and divide this by the total possible conflicts. This ensures that the fitness score remains between 0 and 1. I noticed that this approach was common in online resources, and it proved to be more effective in my code, allowing for a consistent metric that enhances comprehension.

I will provide further details on each aspect below.


In [70]:

class geneticAlgorithm:
    def __init__(self,pop_size, board_size):
        self.board_size = board_size
        self.board = []
        self.pop_size = pop_size
        self.population=[np.array(random.sample(range(self.board_size),self.board_size)) for i in range(pop_size)]
        self.fitness_arr = [ 0 for i in range(self.pop_size)]
        self.fitness_array_function()

    def sort_pop_by_fitness(self):
        n = self.pop_size
        swapped = False
        for i in range(n):
            for j in range(0,n-i-1):
                if self.fitness_arr[j] > self.fitness_arr[j + 1]:
                    swapped = True
                    self.fitness_arr[j], self.fitness_arr[j + 1] = self.fitness_arr[j + 1], self.fitness_arr[j]
                    self.population[j], self.population[j + 1] = self.population[j + 1], self.population[j]
            if not swapped:
                return

    def fitness_function(self,chromosome):
        number_of_conflicts=0
        for i in range(self.board_size):
            for j in range(i+1,self.board_size):
                if chromosome[i] == chromosome[j]:
                    number_of_conflicts += 1
                if abs(i-j) == abs(chromosome[i]-chromosome[j]):
                    number_of_conflicts += 1
        return 1- ( number_of_conflicts / ( self.board_size*(self.board_size - 1)/2 ) )
        
    def fitness_array_function(self):
        for ind in range(self.pop_size):
            current_fitness = self.fitness_function(self.population[ind])
            self.fitness_arr[ind] = current_fitness
        return self.fitness_arr

    def crossover(self,chromosome_A, chromosome_B):
        n = len(chromosome_A)
        point1 = random.randint(0, n-2)
        point2 = random.randint(point1+1, n-1)
        child1 = [None]*n
        child2 = [None]*n

        child1[point1:point2] = chromosome_A[point1:point2]
        child2[point1:point2] = chromosome_B[point1:point2]


        for i in range(n):
            if child1[i] == None:
                if chromosome_B[i] not in child1:
                    child1[i] = chromosome_B[i]
            if child2[i] == None:
                if chromosome_A[i] not in child2:
                    child2[i] = chromosome_A[i]           


        for i in range(n):
            if child1[i] == None:
                for j in range(n):
                    if j not in child1:
                        child1[i] = j
            if child2[i] == None:
                for j in range(n):
                    if j not in child2:
                        child2[i] = j


        child1_fitness=self.fitness_function(child1)
        child2_fitness=self.fitness_function(child2)
        if child1_fitness>child2_fitness:
            return child1
        return child2


    def mutation(self,chromosome,mutation_prob ):
        
        n = len(chromosome)
        if  random.random()<mutation_prob:
            number_of_mutation = random.randint(0,int(0.2*self.board_size))
            for _ in range (number_of_mutation):
                index1 = random.randint(0,self.board_size-1)
                index2 = random.randint(0,self.board_size-1)
                chromosome[index1],chromosome[index2] = chromosome[index2],chromosome[index1] 
        return chromosome

    def select(self):
        new_generation=[]
        for i in range(0,self.pop_size ):
            random_candidates = np.array(random.sample(range(self.pop_size),20))
            max_index1 = max(random_candidates, key=lambda j: self.fitness_arr[j])
            first_parent = (max_index1,self.population[max_index1],self.fitness_arr[max_index1])
            random_candidates = np.array(random.sample(range(self.pop_size),20))
            max_index2 = max(random_candidates, key=lambda j: self.fitness_arr[j])
            second_parent = (max_index2,self.population[max_index2],self.fitness_arr[max_index2])
            new_child = self.crossover(first_parent[1], second_parent[1])
            new_child = self.mutation(new_child,0.8)
            new_generation.append(new_child)
        new_generation.sort(key=lambda item:self.fitness_function(item))
        self.sort_pop_by_fitness()
        for index in range(len(new_generation)):
            if self.fitness_function(new_generation[index]) > self.fitness_arr[index] : 
                self.population[index] = new_generation[index]
        self.fitness_arr=self.fitness_array_function()
            
    


        
    def print_board (self):
        self.sort_pop_by_fitness()
        arr= self.population[-1]
        for row in arr:
            r = ["[ ]"]*len(arr)
            r[row] = "[Q]"
            print ("".join(r))


Translate it into professional english

It responds very quickly for eight, which is quite unusual. This may be due to the fact that I provided my population with more than the number of conflict cases.

In [62]:
n = 8
pop = 48
board = geneticAlgorithm(pop,n)
while(max(board.fitness_arr)<1):
    board.select()
board.print_board()

[4, 7, 3, 0, 6, 1, 5, 2]
[ ][ ][ ][ ][Q][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][Q]
[ ][ ][ ][Q][ ][ ][ ][ ]
[Q][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][Q][ ]
[ ][Q][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][Q][ ][ ]
[ ][ ][Q][ ][ ][ ][ ][ ]


This also converges very quickly, despite the population size being smaller than the number of possible conflict states!

In [60]:
n = 20
pop = 48
board = geneticAlgorithm(pop,n)
while(max(board.fitness_arr)<1):
    board.select()
board.print_board()

[14, 10, 6, 2, 11, 15, 17, 19, 4, 7, 0, 8, 1, 5, 16, 18, 9, 12, 3, 13]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q]
[ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ]
[ ][ ][ ][ ][ 

From this point, the times start to increase a bit; it becomes somewhat evident and noticeable. In this case, I wasn't getting results with a population of 30, but when I increased the population to 100, it improved.

In [64]:
n = 40
pop = 100
board = geneticAlgorithm(pop,n)
while(max(board.fitness_arr)<1):
    board.select()
board.print_board()

[35, 9, 20, 5, 7, 34, 15, 13, 28, 19, 22, 38, 26, 30, 1, 39, 14, 10, 23, 32, 4, 2, 12, 6, 24, 36, 18, 31, 29, 17, 37, 0, 8, 27, 25, 21, 11, 16, 33, 3]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ 

For 60, it suddenly became a lot! But the population worked well, and I had to increase the mutation rate. In the first few runs, it took a long time to reach a result, around 4 minutes, but as I increased the population and the mutation rate, it improved.
A mutation rate of 0.7 is very good for it.

In [66]:
n = 60
pop = 120
board = geneticAlgorithm(pop,n)
maxfir=0
while(max(board.fitness_arr)<1):

    board.select()
board.print_board()

[45, 47, 29, 43, 46, 9, 1, 13, 36, 44, 24, 27, 4, 16, 26, 20, 2, 34, 14, 18, 5, 15, 25, 38, 57, 35, 23, 56, 49, 53, 48, 54, 10, 59, 22, 33, 37, 28, 3, 52, 12, 30, 51, 0, 55, 21, 8, 17, 50, 39, 58, 7, 32, 6, 41, 19, 31, 40, 42, 11]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]

This was really tough 😂 It took a long time to get a result, but finally, it did. I set the mutation rate to 0.8 this last time, and it improved the results. It's surprising how good the fitness is found in our initial population!

In [69]:
n = 100
pop = 200
board = geneticAlgorithm(pop,n)
maxfir=0
while(max(board.fitness_arr)<1):
    print(max(board.fitness_arr))
    board.select()
board.print_board()

0.9918325631100707
0.9928426641201716
0.9930446843221918
0.9934487247262322
0.9942568055343131
0.9944588257363333
0.9956709469484545
0.9958729671504747
0.9964790277565353
0.9968830681605757
0.9970850883625959
0.9970850883625959
0.9972871085646161
0.9974891287666363
0.9974891287666363
0.9974891287666363
0.9976911489686565
0.9976911489686565
0.9978931691706767
0.9980951893726969
0.9984992297767373
0.9984992297767373
0.9987012499787575
0.9987012499787575
0.9987012499787575
0.9987012499787575
0.9987012499787575
0.9989032701807777
0.9991052903827979
0.9991052903827979
0.9991052903827979
0.9991052903827979
0.9991052903827979
0.9991052903827979
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9995093307868383
0.9995093307868383
0.9995093307868383
0.9995093307868383
0.9995093307868383
0.9995093307868383
0.9995093307